# One Particle

In [4]:
!python -V

Python 3.11.8


In [20]:
import matplotlib.pyplot as plt
import numpy as np
from math import *
from uncertainties import *
from scipy.stats import chi2
import scipy
from matplotlib import gridspec
import matplotlib
import pandas
import sys
import statsmodels.api as sm
import warnings ## statsmodels.api is too old ... -_-#

import pickle
import pgzip
import os
import platform

from joblib import Parallel, delayed
from tqdm.notebook import tqdm
from datetime import datetime

import pyfftw

%config InlineBackend.figure_format = 'retina'
%matplotlib inline
plt.rcParams["figure.figsize"] = (8, 5)
plt.rcParams["font.family"] = "serif" 
plt.rcParams["mathtext.fontset"] = "dejavuserif" 
plt.close("all") # close all existing matplotlib plots

In [7]:
nx = 260+1
nz = 90+1
xmax = 26 #Micrometers
# zmax = (nz/nx)*xmax
zmax = 26
dt = 5e-4 # Milliseconds
dx = 2*xmax/(nx-1)
dz = 2*zmax/(nz-1)
hb = 63.5078 #("AtomicMassUnit" ("Micrometers")^2)/("Milliseconds")
m3 = 3   # AtomicMassUnit
m4 = 4 

print("zmax =", zmax)
print("(dx,dz) =", (dx, dz) ) 
print("rotate phase =", 1j*hb*dt/(2*m3*dx*dz)) #want this to be small

pxmax= (nx+1)/2 * 2*pi/(2*xmax)*hb # want this to be greater than p
pzmax= (nz+1)/2 * 2*pi/(2*zmax)*hb
print("pxmax =", pxmax)
print("pzmax =", pzmax)

print(round(nx*nz/1000/1000,3),"million grid points")
print(round((nx*nz)*0.001*0.001/60, 2),"minutes/grid_op (for 1μs/element_op)")

zmax = 26
(dx,dz) = (0.2, 0.5777777777777777)
rotate phase = 0.04579889423076923j
pxmax = 1005.2518680100052
pzmax = 352.98920556076513
0.024 million grid points
0.0 minutes/grid_op (for 1μs/element_op)


In [30]:
use_cache = False
save_cache = False
save_debug = True 

datetime_init = datetime.now()

# os.makedirs('output/oneParticleSim', exist_ok=True)
output_prefix = "output/oneParticleSim/"+\
                datetime_init.strftime("%Y%m%d-%H%M%S") + "-" + \
                str("T" if use_cache else "F") + \
                str("T" if save_cache else "F") + \
                str("T" if save_debug else "F") + \
                "/"
output_ext = ".pgz.pkl"
os.makedirs(output_prefix, exist_ok=True)
print(output_prefix)

output/oneParticleSim/20240416-135603-FFT/


In [31]:
if save_debug:
    with open(output_prefix + "session_info.txt", "wt") as file:
        string = ""
        string += ("="*20 + "Session System Information" + "="*20) + "\n"
        uname = platform.uname()
        string +=  f"Python Version: {platform.python_version()}\n"
        string +=  f"Platform: {platform.platform()}\n"
        string += (f"System: {uname.system}")+ "\n"
        string += (f"Node Name: {uname.node}")+ "\n"
        string += (f"Release: {uname.release}")+ "\n"
        string += (f"Version: {uname.version}")+ "\n"
        string += (f"Machine: {uname.machine}")+ "\n"
        string += (f"Processor: {uname.processor}")+ "\n"
        string +=  f"CPU Counts: {os.cpu_count()} \n"
    #     print(string)
        file.write(string)

In [32]:
wavelength = 1.083 #Micrometers
k = (1/sqrt(2)) * 2*pi / wavelength # effective wavelength
print("k =", k, " is 45 degree Bragg")
# k = 2*pi / wavelength
# k = pi / (2*dz)
k = pi / (4*dx)
# dopd = 60.1025 # 1/ms Doppler detuning (?)

p = 2*hb*k
print("k  =",k,"1/µm")
print("p  =",p, "u*µm/ms")
v4 = hb*k/m4
v3 = hb*k/m3
print("v3 =",v3, "µm/ms")
print("v4 =",v4, "µm/ms")

# sanity check
# assert (pxmax > p*2.5 or pzmax > p*2.5), "momentum resolution too small"
assert (pxmax > p*2.5), "momentum resolution too small"

k = 4.102384984449092  is 45 degree Bragg
k  = 3.9269908169872414 1/µm
p  = 498.78909481412467 u*µm/ms
v3 = 83.13151580235412 µm/ms
v4 = 62.34863685176558 µm/ms
The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


AssertionError: momentum resolution too small